In [13]:
# Импорт модулей
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd

# Скачайте только раз, если не скачивали ранее
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^а-яёa-z0-9\s]', ' ', text)
    words = text.split()
    words = [w for w in words if w not in stop_words]
    return ' '.join(words)

# Загрузите обращения
with open('../data/raw/Obrashcheniia.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

# Очистите обращения
cleaned = [preprocess_text(line) for line in lines]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\masha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1,2), max_features=1000)
X = vectorizer.fit_transform(cleaned)
sums = X.sum(axis=0)
words_freq = [(word, sums[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)

for word, freq in words_freq[:50]:
    print(f"'{word}': {freq}")

'прошу': 167
'10': 161
'требуется': 124
'ошибка': 98
'работает': 95
'1с': 83
'нужна': 67
'заявка': 60
'могу': 60
'оформить': 59
'добавить': 54
'нужно': 53
'документ': 48
'формируется': 47
'11': 47
'заявке': 47
'прошу оформить': 46
'ндс': 45
'документы': 44
'доступ': 42
'ошибку': 42
'настроить': 40
'повторно': 40
'помощь': 40
'12': 39
'16': 38
'выдать': 37
'логи': 35
'срочно': 33
'права': 33
'системе': 32
'error': 30
'работа': 30
'01': 29
'заявки': 28
'проходит': 28
'обновления': 28
'15': 28
'всё': 27
'отпуск': 27
'работы': 25
'16 10': 25
'период': 25
'приложил': 25
'обращение': 25
'срок': 25
'сетевой': 24
'попытке': 24
'работать': 24
'диск': 24


In [15]:
import json
import pandas as pd

# 1. Загрузить файл правил из папки data/rules
with open('../data/rules/rules.json', 'r', encoding='utf-8') as f:
    rules = json.load(f)

# 2. Функция для назначения категории на основе правил
def assign_category(text):
    # Для каждой категории и списка ключевых слов
    for category, keywords in rules.items():
        # Проверяем, есть ли в тексте любое ключевое слово
        for kw in keywords:
            if kw in text:
                return category
    # Если ни одно ключевое слово не найдено, категория 'other'
    return 'other'

# 3. Применить функцию ко всем очищенным текстам
# Замените 'cleaned' на ваш список очищенных обращений
categories = [assign_category(text) for text in cleaned]

# 4. Создать DataFrame из текста и категорий
df = pd.DataFrame({
    'text_clean': cleaned,
    'category': categories
})

# 5. Сохранить DataFrame в CSV для последующего обучения модели
df.to_csv('../data/processed/training_data.csv', index=False)

print("Готово! Ваши данные разметились по категориям и сохранены в training_data.csv")


Готово! Ваши данные разметились по категориям и сохранены в training_data.csv
